# Get trending topics from Google Search

### env config

In [1]:
import os
import sys
import ipykernel

# from dotenv import load_dotenv
# load_dotenv()  # this loads the .env script for use below

PROJECT_ID = os.getenv("PROJECT_ID")
LOCATION = os.getenv("LOCATION")
PREFIX = os.getenv("PREFIX")

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"LOCATION: {LOCATION}")
print(f"PREFIX: {PREFIX}")

PROJECT_ID: hybrid-vertex
LOCATION: us-central1
PREFIX: zghost_v1


### imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from google.cloud import bigquery
import pandas as pd
import numpy as np
# from datetime import datetime, timedelta

bq = bigquery.Client(project = PROJECT_ID)

## BigQuery public dataset

In [5]:
TARGET_DATE = "03/25/2025"

In [6]:
query = f"""
    SELECT
        term,
        refresh_date,
        ARRAY_AGG(STRUCT(rank,week) ORDER BY week DESC LIMIT 1) x
    FROM 
        `bigquery-public-data.google_trends.top_terms`
    WHERE
        refresh_date = PARSE_DATE('%m/%d/%Y',  '{TARGET_DATE}')
    GROUP BY    
        term, refresh_date
    ORDER BY
        (SELECT rank FROM UNNEST(x))
"""
print(query)


    SELECT
        term,
        refresh_date,
        ARRAY_AGG(STRUCT(rank,week) ORDER BY week DESC LIMIT 1) x
    FROM 
        `bigquery-public-data.google_trends.top_terms`
    WHERE
        refresh_date = PARSE_DATE('%m/%d/%Y',  '03/25/2025')
    GROUP BY    
        term, refresh_date
    ORDER BY
        (SELECT rank FROM UNNEST(x))



In [15]:
target_trends_df = bq.query(query).to_dataframe()
target_trends_df.head(10)

,term,refresh_date,x
0,bolivia vs uruguay,2025-03-25,"[{'rank': 1, 'week': 2025-03-23 00:00:00}]"
1,bolivia - uruguay,2025-03-25,"[{'rank': 2, 'week': 2025-03-23 00:00:00}]"
2,jasmine crockett,2025-03-25,"[{'rank': 3, 'week': 2025-03-23 00:00:00}]"
3,troy taylor,2025-03-25,"[{'rank': 4, 'week': 2025-03-23 00:00:00}]"
4,outside lands 2025,2025-03-25,"[{'rank': 5, 'week': 2025-03-23 00:00:00}]"
5,aaliyah chavez,2025-03-25,"[{'rank': 6, 'week': 2025-03-23 00:00:00}]"
6,is chipotle closing,2025-03-25,"[{'rank': 7, 'week': 2025-03-23 00:00:00}]"
7,erik per sullivan,2025-03-25,"[{'rank': 8, 'week': 2025-03-23 00:00:00}]"
8,nigeria vs zimbabwe,2025-03-25,"[{'rank': 9, 'week': 2025-03-23 00:00:00}]"
9,gme stock,2025-03-25,"[{'rank': 10, 'week': 2025-03-23 00:00:00}]"


In [ ]:
sample_term = target_trends_df['term'].iloc[0]
print(sample_term)

bolivia vs uruguay
